In [31]:
"""
Benchmark link : https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/JMFHTN

"""

import re
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plot
import numpy as np

def save_obj(obj, name ,overwrite=1):
	filename='data/'+ name + '.pkl';
	if(overwrite==1 and os.path.exists(name)):
		return [];
	with open(filename, 'wb') as f:
		pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
	filename='data/'+ name + '.pkl';
	# if(not os.path.exists(name)):
	# 	return [];
	with open('data/' + name + '.pkl', 'rb') as f:
		return pickle.load(f)

seq_records =pd.read_table("./data/uniprot-all.tab.gz", sep='\t')


In [32]:
seq_records.sample(10)

,Entry,Entry name,Sequence,Protein families
368263,A9KD24,RL16_COXBN,MLQPSNRKYRKDFKGRNRGVASRGNRVSFGEFGLKATECARITARQ...,Universal ribosomal protein uL16 family
34490,A4XKX1,ATPG_CALS8,MANKTRWIKSRIKSVNETKKITRAMYLISASKMKRSRDRLDSTRPY...,ATPase gamma chain family
535578,C0MHH2,Y1222_STRS7,MSDKQINLVIVTGMSGAGKTVAIQSFEDLGYFTVDNMPPALVPKFL...,RapZ-like family
40204,P84729,BAS1_PINST,GLFIIDKEGVIQHSTINNEGVIQHSTINNLAIGRFGVLLADQGLAL...,AhpC/TSA family
79716,P0DJB5,CUX4_CONCL,RPKCCCVCGVVGRKCCSTWDKCHPVHLPCPSS,NaN
13450,C7GJG9,AIM21_YEAS2,MSSEVTPKVPERPSRRKTSELFPLSGSESGDIEANSEPPTPAGTPN...,AIM21 family
379616,Q3J8S5,RL24_NITOC,MRRVRVGDEVIVTAGRSKGKQGKILRILGDERVIVQDVNMVKRHTR...,Universal ribosomal protein uL24 family
296031,P85349,PER4_CYCRE,YYVDLQNR,"Peroxidase family, Classical plant (class III)..."
289265,Q9FVV9,NAP5_ARATH,MDRERYDKVIEACSLSKDLEILSFGDQTVIGERGINLSGGQKQRIH...,ABC transporter superfamily
464535,Q4KMQ1,TPRN_HUMAN,MAALGRPGSGPRAAVPAWKREILERKRAKLAALGGGAGPGAAEPEQ...,Taperin family


In [27]:
%matplotlib inline
seq_records[seq_records['Sequence'].str.contains('X|U|B')].sample(10)

,Entry,Entry name,Sequence,Protein families
534318,Q9PKX0,Y341_CHLMU,MLSCIFKDTIFLSSFLAVSLICMTTALWGTILLVERRPLLSESLSH...,ABC-3 integral membrane protein family
314665,Q9MUK0,PSAA_CYCRE,IVVEMDPIKTSFEKWAKPGHFSXTLXKGPNTTTWIWNLHADAHDFG...,PsaA/PsaB family
425916,P59797,SELV_HUMAN,MNNQARTPAPSSARTSTSVRASTPTRTPTPLRTPTPVRTRTPIRTL...,SelWTH family
88734,Q33754,CYB_ABRLO,MTIMRKTHPLLKIINHSFIDLPTPSNISSWWNFGSLLGICLIIQIL...,Cytochrome b family
120078,B5E488,ERA_STRP4,MTFKSGFVAILGRPNVGKSTFLNHVMGQXIAIMXDKAXTTRNKIMG...,TRAFAC class TrmE-Era-EngA-EngB-Septin-like GT...
396583,P80896,RPN1_CHICK,VFTHVLQPYPEIRPRFPLVGGWKTHYXV,OST1 family
428087,O51596,SECD_BORBU,MKKGSKLILILLVTFFACLLIFPTLKWYFLMSVEDKKISSYSQEAL...,"SecD/SecF family, SecD subfamily"
20351,P10748,AROA_SOLLC,MAQISSMAQGIQTLSLNSSNLSKTQKGPLVSNSLFFGSKKLTQISA...,EPSP synthase family
440177,P34936,SODC_HORVU,MXKAVAVLTGSEGVXGTIFFTQ,Cu-Zn superoxide dismutase family
270441,P97700,M2OM_RAT,MAATASPGAGRMDGKPRTSPKSVKFLFGGLAGMGATVFVQPLDLVX...,Mitochondrial carrier (TC 2.A.29) family


In [2]:
i=1
num_recs=40
col = 'Protein families'
value_counts = seq_records[col].value_counts()
to_remove = value_counts[value_counts <= 200].index
filtered = seq_records[col].replace(to_remove, np.nan, inplace=False)
# print(seq_records.describe())
# filtered.describe()
# seq_records.groupby(['Protein families'], as_index=False).describe()


In [3]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [4]:
class myLSTM():
    def __init__(self,alpha,num_classes=8,mval=0.5):
        #         All main code here
        num_features=42
        depth = 3-1
        #         init  Weights
        self.alpha=alpha

        self.weights = tf.Variable(tf.random_uniform([hidden_units*depth, num_classes],minval=-mval,maxval=mval),name='Weights')
        #         init  X & Y
        shapeX=[None,seqLen,num_features]
        shapeY=[None,seqLen,num_classses]
        # Placeholders - Used in feed_dicts
        self.pholder_x= tf.place_holder('float',shapeX,name="InputData") 
        self.pholder_y= tf.place_holder('float',shapeY,name="LabelData")

        #Now make model here -
        with tf.name_scope('Model'):
            self.cell1 = rnn.LSTMCell(num_hidden_units)

        #The Loss function -
        with tf.name_scope('Loss'):
            self.cost = tf.nn.softmax_cross_entropy_with_logits(logits=self.pred_y,labels=y)
        #The Grad Optimiser with minimize -
        with tf.name_scope('GradDescent'):
            self.trainer = tf.train.GradientDescentOptimizer(alpha).minimize(self.loss_fn)
        #Accuracy calcs -
        with tf.name_scope('Accuracy'):
            self.acc = tf.reduce_mean(tf.cast(
                tf.equal(
                    tf.argmax(self.y,1),
                    tf.argmax(self.pred_y,1)
                    ),
                tf.float32))
        #         So, all declared now initialize
        self.sess = tf.Session()
        self.init = tf.global_variables_initializer()
        self.sess.run(self.init) #later might move somewhere.

        # Create a summary to monitor cost tensor
        tf.summary.scalar("loss", self.cost)
        # Create a summary to monitor accuracy tensor
        tf.summary.scalar("accuracy", self.acc)
        # Merge all summaries into a single op - to pass into sess.run
        merged_summary_op = tf.summary.merge_all()


    def predict(self, x, y):
        result = self.sess.run(self.y_predicted, feed_dict={self.x : x, self.y : y})
        return result

    def summary_writer(logs_path):
        # op to write logs to Tensorboard
        return tf.summary_writer.FileWriter(logs_path, graph=tf.get_default_graph())
        

    def cross_validate(self, x, y):
        result = self.sess.run(self.accuracy, feed_dict={self.x : x, self.y : y})
        return result
    # first arg to sess.run is what component shoud run.
    
    def  train(self,x,y):
            
        _, c, summary = sess.run([self.trainer, self.cost, self.merged_summary_op],
                                 feed_dict={ self.pholder_x :x ,self.pholder_y:y })
        return (c,summary)


In [ ]:
training_epochs=
batch_size=
display_step=10
logs_path='/tmp/tensorflow/logs'
num_batches = y.shape[0] / batch_size
# Training cycle
summary_writer = model.summary_writer(logs_path)
for epoch in range(training_epochs):
    avg_cost = 0.

    # Loop over all batches
    for i in range(num_batches):
        batch_xs, batch_ys = #??tf.train.next_batch(batch_size)
        # Run optimization op (backprop), cost op (to get loss value)
        # and summary nodes
        c,summary = model.train(batch_xs,batch_ys)
        # Write logs at every iteration
        summary_writer.add_summary(summary, epoch * num_batches + i)
        # Compute average loss
        avg_cost += c / num_batches
    # Display logs per epoch step
    if (epoch+1) % display_step == 0:
        print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)

print "Optimization Finished!"
print "Accuracy:", self.acc.eval({x: self.test_x, y: self.test_y})
